In [1]:
config = {
    'host': 'localhost',
    'port' : 5432,
    'database': 'mimic_iv',
    'user': 'mimic_read_only_user',
    'password': 'mimiciv',
    'schema': 'hosp'
}

from sqlalchemy import create_engine, text
import pandas as pd



query = """
SELECT 
    p.*,
    CASE 
        WHEN matches[1] IS NOT NULL THEN CAST(matches[1] AS FLOAT)
        ELSE NULL
    END AS quantity,
    CASE 
        WHEN matches[1] IS NOT NULL THEN matches[3]
        ELSE NULL
    END AS unit,
    CASE 
        WHEN matches[1] IS NOT NULL THEN TRIM(BOTH ' ' FROM matches[5])
        ELSE p.prod_strength
    END AS form
FROM 
    hosp.prescriptions p,
    LATERAL REGEXP_MATCHES(p.prod_strength, '^([0-9]+(\.[0-9]+)?)?[ ]?([a-zA-Z]+(/[a-zA-Z]+)?)?([a-zA-Z ]+)$') AS matches
LIMIT 200000
    """





engine = create_engine(f'postgresql://{config["user"]}:{config["password"]}@{config["host"]}:{config["port"]}/{config["database"]}')
with engine.connect() as connection:
    df = pd.read_sql_query(text(query), connection)
df



<>:15: SyntaxWarning: invalid escape sequence '\.'
<>:15: SyntaxWarning: invalid escape sequence '\.'
C:\Users\jvoyer\AppData\Local\Temp\4\ipykernel_15804\4022692885.py:15: SyntaxWarning: invalid escape sequence '\.'
  query = """
C:\Users\jvoyer\AppData\Local\Temp\4\ipykernel_15804\4022692885.py:15: SyntaxWarning: invalid escape sequence '\.'
  query = """


OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (::1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [2]:
df[df['dose_val_rx'].str.contains('-')]

,subject_id,hadm_id,pharmacy_id,poe_id,poe_seq,order_provider_id,starttime,stoptime,drug_type,drug,...,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route,quantity,unit,form
58,10959054,29665973,13891188,10959054-583,583.0,P91MW8,2186-12-17 22:00:00,2186-12-19 17:00:00,MAIN,Zolpidem Tartrate,...,None,2.5-5,mg,0.5-1,TAB,NaN,PO,5.0,mg,Tablet
63,10959054,29665973,22306696,10959054-561,561.0,P91MW8,2186-12-17 22:00:00,2186-12-20 17:00:00,MAIN,Aluminum-Magnesium Hydrox.-Simethicone,...,None,15-30,mL,0.5-1,UDCUP,NaN,PO,30.0,mL,UDCup
71,10959054,29665973,34646412,10959054-623,623.0,P70N8Q,2186-12-19 09:00:00,2186-12-20 17:00:00,MAIN,TraMADOL (Ultram),...,None,25-50,mg,0.5-1,TAB,NaN,PO,50.0,mg,Tablet
75,10959054,29665973,53335869,10959054-585,585.0,P91MW8,2186-12-17 22:00:00,2186-12-20 17:00:00,MAIN,Albuterol-Ipratropium,...,None,1-2,PUFF,0.01-0.02,CAN,NaN,IH,14.7,g,Inhaler
92,10959054,29665973,87937273,10959054-618,618.0,P90MUK,2186-12-18 21:00:00,2186-12-19 08:00:00,MAIN,OxycoDONE (Immediate Release),...,None,5-10,mg,1-2,TAB,NaN,PO,5.0,mg,Tablet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199967,11140311,27454707,3463504,11140311-74,74.0,P02L7J,2147-05-20 17:00:00,2147-05-23 18:00:00,MAIN,OxyCODONE (Immediate Release),...,None,5-10,mg,1-2,TAB,NaN,PO/NG,5.0,mg,Tablet
199968,11140311,27454707,19097944,11140311-69,69.0,P02L7J,2147-05-20 17:00:00,2147-05-23 18:00:00,MAIN,Acetaminophen,...,None,500-1000,mg,1-2,TAB,NaN,PO,500.0,mg,Tablet
199972,11140311,27454707,42295351,11140311-47,47.0,P86JLW,2147-05-20 17:00:00,2147-05-21 17:00:00,MAIN,Ketorolac,...,None,15-30,mg,1-2,VIAL,NaN,IV,15.0,mg/mL,Vial
199973,11140311,27454707,46458228,11140311-47,47.0,P86JLW,2147-05-20 17:00:00,2147-05-21 17:00:00,MAIN,Ketorolac,...,None,15-30,mg,0.5-1,VIAL,NaN,IV,30.0,mg/mL,Vial


In [3]:
df.loc[df['dose_val_rx'].str.contains('to'), 'dose_val_rx'] = df['dose_val_rx'].str.replace('to', '-')

In [4]:
df['dose_val_rx'] = df['dose_val_rx'].str.strip('-')

In [5]:
#check for ranges in dose_val_rx (e.g. 1-2)*
import numpy as np

df["dose_val_rx"] = df['dose_val_rx'].where(df['dose_val_rx'].str.contains('-'), df['dose_val_rx'].str.split('-').apply(lambda x: [float(i.replace(",",'.')) for i in x]).apply(lambda x: np.mean(x)))

ValueError: could not convert string to float: '1.62 % (4'

In [6]:
df.loc[~df['dose_val_rx'].apply(lambda x : isinstance(x, float)),"dose_val_rx"]=df[~df['dose_val_rx'].apply(lambda x : isinstance(x, float))]["dose_val_rx"].apply(lambda x: x.split('-')).apply(lambda x: [float(i) for i in x]).apply(lambda x: np.mean(x))
df.loc[~df['form_val_disp'].apply(lambda x : isinstance(x, float)),"form_val_disp"]=df[~df['form_val_disp'].apply(lambda x : isinstance(x, float))]["form_val_disp"].apply(lambda x: x.split('-')).apply(lambda x: [float(i) for i in x]).apply(lambda x: np.mean(x))

In [7]:

# check if the dose_unit_rx is a milli
df["dose_val_rx"]=df["dose_val_rx"].astype(float)
records = df.to_dict(orient='records')
for record in records:
    if isinstance(record['dose_unit_rx'], str):
        record['dose_unit_rx'] = record['dose_unit_rx'].strip()
        if record['dose_unit_rx'].startswith('m'):
            record['dose_val_rx'] = record['dose_val_rx'] / 1000
            record['dose_unit_rx'] = record['dose_unit_rx'][1:]
    if isinstance(record['form_unit_disp'], str):
        record['form_unit_disp'] = record['form_unit_disp'].strip()
        if record['form_unit_disp'].startswith('m'):
            record['form_val_disp'] = record['form_val_disp'] / 1000
            record['form_unit_disp'] = record['form_unit_disp'][1:]
            
    if isinstance(record['unit'], str):
        record['unit'] = record['unit'].strip()
        if record['unit'].startswith('m'):
            record['quantity'] = record['quantity'] / 1000
            record['unit'] = record['unit'][1:]
            if record['unit'] == 'l' : record['unit'] = 'L'
            if record['unit'] == 'G' : record['unit'] = 'g'
        if "/" in record['unit']:
            nom, denom = record['unit'].split('/')
            if denom.strip().startswith('m'):
                record['quantity'] = record['quantity'] * 1000
                denom = denom[1:]
            if denom == 'l' : denom = 'L'
            if denom == 'G' : denom = 'g'
            if nom == 'l' : nom = 'L'
            if nom == 'G' : nom = 'g'
            new_unit = nom + '/' + denom
            record['unit'] = new_unit
        
df = pd.DataFrame(records)
df[['drug','dose_val_rx', 'dose_unit_rx', 'quantity', 'unit', 'form', 'prod_strength']]


,drug,dose_val_rx,dose_unit_rx,quantity,unit,form,prod_strength
0,5% Dextrose,0.050,L,0.050,L,Bag,50 mL Bag
1,Ibuprofen,0.500,g,0.600,g,Tablet,600mg Tablet
2,Potassium Chloride,0.040,Eq,0.010,Eq,ER Tablet,10mEq ER Tablet
3,5% Dextrose,0.100,L,0.100,L,Bag,100 mL Bag
4,Doxycycline Hyclate,0.100,g,0.100,g,Vial,100 mg Vial
...,...,...,...,...,...,...,...
199995,D5 1/2NS,1.000,L,1.000,L,Bag,1000mL Bag
199996,Acetaminophen,0.650,g,0.325,g,Tablet,325mg Tablet
199997,Ondansetron,0.004,g,0.004,g,Tablet,4 mg Tablet
199998,Ibuprofen,0.800,g,0.400,g,Tablet,400mg Tablet


In [8]:
df.keys()   

Index(['subject_id', 'hadm_id', 'pharmacy_id', 'poe_id', 'poe_seq',
       'order_provider_id', 'starttime', 'stoptime', 'drug_type', 'drug',
       'formulary_drug_cd', 'gsn', 'ndc', 'prod_strength', 'form_rx',
       'dose_val_rx', 'dose_unit_rx', 'form_val_disp', 'form_unit_disp',
       'doses_per_24_hrs', 'route', 'quantity', 'unit', 'form'],
      dtype='object')

In [9]:
import seaborn as sns
way= df.groupby(['gsn'])[["unit","form"]].nunique()
way



,unit,form
gsn,,
,5,6
000018,1,1
000019,1,1
000090,1,1
000091,1,1
...,...,...
078262,1,1
078733,1,1
078791,1,1


> On va travailler sur les références de médicaments présentant 1 seule forme et 1 seule unité

In [10]:
way[(way['unit']==1) & (way['form']==1)]

,unit,form
gsn,,
000018,1,1
000019,1,1
000090,1,1
000091,1,1
000093,1,1
...,...,...
078262,1,1
078733,1,1
078791,1,1


In [11]:
work_df = df[df['gsn'].isin(way[(way['unit']==1) & (way['form']==1)].index)]

work_df

,subject_id,hadm_id,pharmacy_id,poe_id,poe_seq,order_provider_id,starttime,stoptime,drug_type,drug,...,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route,quantity,unit,form
1,10791629,21819658,46855320,10791629-107,107.0,P83VKT,2171-01-16 15:00:00,2171-01-16 15:00:00,MAIN,Ibuprofen,...,None,0.500,g,0.83335,TAB,NaN,PO,0.600,g,Tablet
2,10791629,21819658,55268241,10791629-94,94.0,P83VKT,2171-01-15 19:00:00,2171-01-16 21:00:00,MAIN,Potassium Chloride,...,None,0.040,Eq,4.00000,TAB,1.0,PO,0.010,Eq,ER Tablet
4,10791629,21819658,58680366,10791629-87,87.0,P37BV6,2171-01-15 17:00:00,2171-01-16 11:00:00,MAIN,Doxycycline Hyclate,...,,0.100,g,1.00000,VIAL,2.0,IV,0.100,g,Vial
6,10791629,21819658,63026755,10791629-102,102.0,P83VKT,2171-01-16 12:00:00,2171-01-16 21:00:00,MAIN,Clindamycin,...,None,0.450,g,3.00000,CAP,4.0,PO,0.150,g,Cap
7,10791629,21819658,63405608,10791629-47,47.0,P75HZC,2171-01-14 11:00:00,2171-01-15 16:00:00,MAIN,Ibuprofen,...,None,0.600,g,1.00000,TAB,4.0,PO,0.600,g,Tablet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,10958987,28465678,96525879,10958987-41,41.0,P408EG,2124-12-22 19:00:00,2124-12-24 09:00:00,BASE,D5 1/2NS,...,None,1.000,L,1.00000,L,NaN,IV,1.000,L,Bag
199996,10959007,21338482,642943,10959007-92,92.0,P743BY,2118-10-25 02:00:00,2118-10-26 22:00:00,MAIN,Acetaminophen,...,None,0.650,g,2.00000,TAB,NaN,PO/NG,0.325,g,Tablet
199997,10959007,21338482,5851476,10959007-96,96.0,P743BY,2118-10-25 02:00:00,2118-10-26 22:00:00,MAIN,Ondansetron,...,None,0.004,g,1.00000,TAB,NaN,PO/NG,0.004,g,Tablet
199998,10959007,21338482,9918114,10959007-115,115.0,P44Z0T,2118-10-26 12:00:00,2118-10-26 22:00:00,MAIN,Ibuprofen,...,None,0.800,g,2.00000,TAB,NaN,PO,0.400,g,Tablet


In [20]:
work_df.loc[:,'dispensation_qty'] = work_df.where(work_df['dose_unit_rx'] == work_df['unit'], work_df['dose_val_rx'] / work_df['quantity'], axis=0)

> On va vérifier si le champ_form_val_disp est bien renseigné

In [21]:
work_df['dispensation_delta'] =work_df['form_val_disp'] - work_df['dispensation_qty']
work_df
work_df[(work_df['form_val_disp'] - work_df['dispensation_qty'])!=0][['prod_strength','dose_val_rx','dose_unit_rx','quantity','unit','form','form_val_disp','form_unit_disp','dispensation_qty','dispensation_delta']]

C:\Users\jvoyer\AppData\Local\Temp\5\ipykernel_14232\953967689.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  work_df['dispensation_delta'] =work_df['form_val_disp'] - work_df['dispensation_qty']


,prod_strength,dose_val_rx,dose_unit_rx,quantity,unit,form,form_val_disp,form_unit_disp,dispensation_qty,dispensation_delta
1,600mg Tablet,0.500,g,0.6000,g,Tablet,0.83335,TAB,0.833333,0.000017
18,12g Inhaler,2.000,PUFF,12.0000,g,Inhaler,0.02000,INH,0.166667,-0.146667
25,8 g Inhaler,1.500,PUFF,8.0000,g,Inhaler,0.01500,INH,0.187500,-0.172500
33,16g NASAL SPRAY,2.000,SPRY,16.0000,g,NASAL SPRAY,0.02000,AERO,0.125000,-0.105000
36,50mL Syringe,12.500,gm,0.0500,L,Syringe,0.50000,SYR,250.000000,-249.500000
...,...,...,...,...,...,...,...,...,...,...
199911,2.5mL Vial,1.000,NEB,0.0025,L,Vial,1.00000,VIAL,400.000000,-399.000000
199939,2.5mL Vial,1.000,NEB,0.0025,L,Vial,1.00000,VIAL,400.000000,-399.000000
199970,1000mL Bag,1.000,L,1.0000,L,Bag,0.00100,L,1.000000,-0.999000
199979,50mg/mL Vial,0.025,g,50.0000,g/L,Vial,0.50000,VIAL,0.000500,0.499500


# Cas spécifique : Hepatite C chronique sans mention de coma hépatique

In [2]:
sql_query = """
SELECT 'SELECT subject_id, hadm_id, ' ||
       STRING_AGG(DISTINCT 'SUM(CASE WHEN gsn = ''' || gsn || ''' THEN 1 ELSE 0 END) AS "GSN_' || gsn || '"', ', ')
       || ' FROM (select p.* from prescriptions p 
join diagnoses_icd di on p.hadm_id = di.hadm_id
where di.icd_code = ''07054'')sub GROUP BY subject_id, hadm_id ORDER BY subject_id, hadm_id;'
AS pivot_query
FROM (select p.* from prescriptions p 
join diagnoses_icd di on p.hadm_id = di.hadm_id
where di.icd_code = '07054')sub;
"""

engine = create_engine(f'postgresql://{config["user"]}:{config["password"]}@{config["host"]}:{config["port"]}/{config["database"]}')
with engine.connect() as connection:
    df = pd.read_sql_query(text(sql_query), connection)

pivot_query = df['pivot_query'][0]

with engine.connect() as connection:
    df = pd.read_sql_query(text(pivot_query), connection)
df.to_csv('dataset.csv', index=False)

OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (::1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [15]:
import duckdb
import pandas as pd
from tqdm.notebook import tqdm, trange
dataset = pd.read_csv('dataset.csv')
dataset.drop(columns=['subject_id','hadm_id', 'GSN_'], inplace=True)
dataset ['correct'] = True


#generate random mistakes in a duplicate dataset
import random as rd

def generate_mistakes(dataset):
    mistakes_2 = []
    for row in tqdm(dataset.to_dict('records')):
        for _ in range(len(dataset.keys())//20):
            mistake_key = rd.choice(dataset.keys())
            mistake_value = rd.choice(dataset[mistake_key].unique())
            row[mistake_key] = mistake_value
            row['correct'] = False
            mistakes_2.append(row)
    mistakes_2 = pd.DataFrame(mistakes_2)
    return mistakes_2

big_dataset = pd.concat([dataset, generate_mistakes(dataset)], ignore_index=True)

#shuffle the dataset

big_dataset = big_dataset.sample(frac=1).reset_index(drop=True)





  0%|          | 0/5138 [00:00<?, ?it/s]

Codes gsn de sofosbuvir : $[071748,072926,076305,076305]$

code gsn d' amiodarone : $000266$


In [16]:
from sklearn.cluster import KMeans
import numpy as np

classes = ['0', '1', '2+']

def classification (data) :
    for key in tqdm(data.columns):
        if key.startswith('GSN_'):
            data[key] = data[key].apply(lambda x: '0' if x == 0 else '1' if x == 1 else '2+')
    return data


In [17]:
classification(big_dataset)

  0%|          | 0/1598 [00:00<?, ?it/s]

,GSN_000015,GSN_000018,GSN_000019,GSN_000090,GSN_000091,GSN_000141,GSN_000239,GSN_000241,GSN_000265,GSN_000266,...,GSN_071674,GSN_071708,GSN_071748,GSN_072514,GSN_072862,GSN_072864,GSN_072866,GSN_072926,GSN_074020,correct
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,False
1,0,0,0,0,0,2+,0,0,0,0,...,0,0,0,0,0,0,0,0,0,False
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,False
3,0,0,0,0,0,2+,0,0,0,0,...,0,0,0,0,0,0,0,0,0,False
4,0,0,0,0,0,0,0,0,0,2+,...,0,0,0,0,0,0,0,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411035,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,False
411036,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,False
411037,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,False
411038,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,False


In [18]:
train, test = big_dataset[:int(len(big_dataset)*0.7)], big_dataset[int(len(big_dataset)*0.7):]


In [19]:
def getFormulasGranules(column_attr, info_table : pd.DataFrame) :
    basic_granules_table = pd.DataFrame(columns = ['Formula', 'Granule'])

    i = 0
    for attr in tqdm(column_attr) :
        group = info_table.groupby(attr).groups
        keys = list(group.keys())
        values = list(group.values())

        for key in keys :
            basic_granules_table.loc[i, 'Formula'] = str(attr) + ' = ' + str(key)
            basic_granules_table.loc[i, 'Granule'] = list(group[key])
            i += 1

    return basic_granules_table

In [52]:

from tqdm.notebook import tqdm, trange
def getGenerality(basic_gr_table: pd.DataFrame, info_table: pd.DataFrame) :
    U = len(info_table)
    for idx in tqdm(basic_gr_table.index) :
        obj_in_granule = len(basic_gr_table.loc[idx, 'Granule'])
        basic_gr_table.loc[idx, 'Generality'] = obj_in_granule / U

In [53]:
def countClasseGr(objects_Gr : pd.DataFrame, info_table: pd.DataFrame) :
    class_0 = 0
    class_1 = 0
    for obj in objects_Gr :
        if info_table.loc[obj, 'correct'] == True :
            class_1 += 1
        else :
            class_0 += 1
    return class_0, class_1

def getConfidence(basic_gr_table: pd.DataFrame, info_table: pd.DataFrame) :
    for idx in tqdm(basic_gr_table.index) :
        objects_Gr = basic_gr_table.loc[idx, 'Granule']
        class_0, class_1 = countClasseGr(objects_Gr, info_table)
        basic_gr_table.loc[idx, 'confidence_0'] = class_0 / len(objects_Gr)
        basic_gr_table.loc[idx, 'confidence_1'] = class_1 / len(objects_Gr)

In [54]:
def getCoverage(basic_gr_table : pd.DataFrame, info_table: pd.DataFrame) :
    if False in info_table[['correct']].values :
        class_0_count = len(info_table.groupby('correct').groups[0])
    else :
        class_0_count = 0

    if 1 in info_table[['correct']].values :
        class_1_count = len(info_table.groupby('correct').groups[1])
    else :
        class_1_count = 0

    for idx in tqdm(basic_gr_table.index) :
        objects_Gr = basic_gr_table.loc[idx, 'Granule']
        class_0, class_1 = countClasseGr(objects_Gr, info_table)
        basic_gr_table.loc[idx, 'coverage_0'] = class_0 / class_0_count
        basic_gr_table.loc[idx, 'coverage_1'] = class_1 / class_1_count

In [55]:
import numpy as np
def getEntropy(basic_gr_table : pd.DataFrame , info_table : pd.DataFrame) :
    res = 0

    for idx in tqdm(basic_gr_table.index) :
        for j in range(2) :
            p_ = basic_gr_table.loc[idx, 'confidence_' + str(j)]
            if p_ == 0 : 
                res+=0
            else :
                res += -p_ * np.log2(p_)
        basic_gr_table.loc[idx, 'Entropy'] = res
        res = 0

        

### Calcul de complexité : 
#### Taille du tableau d'information

soit $n$ le nombre de prescriptions $\newline$
soit $m$ le nombre de médicaments prescrits sur le dataset

on a un tableau Info_table de taille $n\cdot m$

#### Génération des granules

Ici j'ai regroupé les valeurs de chaque colonne (médicament) en 3 catégories :
 * 0 prescription
 * 1 prescription
 * 2 prescriptions ou plus

On a donc au total $3m$ formules au pire cas

#### Calcul de Généralité : 

Le calcul de généralité parcourt chaque granule pour connaitre sa taille.

Chaque granule est dans le pire des cas de taille $n$.

On a donc en complexité temporelle une fonction de complexité $O(3nm)$

#### Compter les classes d'un groupe d'objets : 

recherche dans info_table : complexité en $O(n)$

pour un échantillon de taille k : 

Décompte en complexité $(O(nk))$


#### Obtention de la confiance des formules : 

on a $3m$ formules de taille $n$ au pire cas.

pour chacune de ces formules on compte les classes du granule en complexité $O(n\cdot n)$ selon la formule précédente.

On a donc une complexité de $O(3m\cdot n^2)$ pour obtenir la confiance des formules.



#### Obtention de la couverture des formules

Comme pour la confiance : complexité en $O(3m \cdot n^2)$

#### Obtention de l'entropie des granules

Parcours des formules puis 1 opération par classe finale

complexité en $O(6m)$

In [41]:
covering_solution = set()

attributes = dataset.keys()[2:-1]


u_info_table = train

print("création des greanules")
u_B_Granules = getFormulasGranules(attributes, u_info_table)

print("calcul de la généralité")
getGenerality(u_B_Granules, u_info_table)

print("calcul de la confiance")
getConfidence(u_B_Granules, u_info_table)

print("calcul de la couverture")
getCoverage(u_B_Granules, u_info_table)

print("calcul de l'entropie")
getEntropy(u_B_Granules, u_info_table)
u_B_Granules

création des greanules


  0%|          | 0/1595 [00:00<?, ?it/s]

calcul de la généralité


  0%|          | 0/4259 [00:00<?, ?it/s]

calcul de la confiance


...

calcul de la couverture


  0%|          | 0/4259 [00:00<?, ?it/s]

calcul de l'entropie


  0%|          | 0/4259 [00:00<?, ?it/s]

,Formula,Granule,Generality,confidence_0,confidence_1,coverage_0,coverage_1,Entropy
0,GSN_000019 = 0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14,...",0.966135,0.987337,0.012663,0.965746,0.997450,0.097968
1,GSN_000019 = 1,"[19, 30, 111, 120, 121, 167, 211, 290, 539, 67...",0.018264,0.998668,0.001332,0.018466,0.001984,0.014645
2,GSN_000019 = 2+,"[12, 115, 125, 142, 159, 346, 353, 389, 393, 5...",0.015602,0.999554,0.000446,0.015788,0.000567,0.005602
3,GSN_000090 = 0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",0.979839,0.987490,0.012510,0.979595,0.999433,0.097010
4,GSN_000090 = 1,"[146, 177, 239, 258, 302, 319, 368, 457, 490, ...",0.020161,0.999655,0.000345,0.020405,0.000567,0.004463
...,...,...,...,...,...,...,...,...
4254,GSN_072926 = 0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 16,...",0.957787,0.987252,0.012748,0.957319,0.995466,0.098502
4255,GSN_072926 = 1,"[12, 88, 117, 319, 352, 391, 393, 486, 531, 64...",0.012269,0.997734,0.002266,0.012393,0.002267,0.023176
4256,GSN_072926 = 2+,"[14, 15, 40, 55, 74, 83, 101, 110, 146, 209, 2...",0.029945,0.999071,0.000929,0.030289,0.002267,0.010692
4257,GSN_074020 = 0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",0.978191,0.987476,0.012524,0.977934,0.998867,0.097098


In [45]:

original = u_B_Granules.copy()

In [46]:
u_B_Granules.sort_values(by =['Entropy', 'Generality'], ascending = [True, False], inplace = True)
u_B_Granules.reset_index(drop = True)
u_B_Granules

,Formula,Granule,Generality,confidence_0,confidence_1,coverage_0,coverage_1,Entropy
8,GSN_000141 = 2+,"[1, 3, 7, 36, 37, 64, 103, 132, 224, 273, 296,...",0.033660,1.000000,0.000000,0.034078,0.000000,0.000000
1502,GSN_007873 = 1,"[43, 71, 85, 155, 180, 184, 218, 272, 279, 290...",0.029243,1.000000,0.000000,0.029606,0.000000,0.000000
3888,GSN_061173 = 1,"[1, 74, 109, 175, 187, 221, 255, 288, 386, 417...",0.028774,1.000000,0.000000,0.029131,0.000000,0.000000
1946,GSN_011682 = 2+,"[58, 113, 144, 202, 245, 279, 282, 322, 341, 3...",0.028339,1.000000,0.000000,0.028691,0.000000,0.000000
529,GSN_003202 = 2+,"[42, 169, 170, 171, 247, 286, 294, 377, 387, 3...",0.028054,1.000000,0.000000,0.028403,0.000000,0.000000
...,...,...,...,...,...,...,...,...
3941,GSN_062823 = 1,"[0, 8, 21, 25, 29, 45, 48, 57, 60, 71, 101, 11...",0.070542,0.987042,0.012958,0.070493,0.074525,0.099817
610,GSN_003753 = 0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 15,...",0.822579,0.987025,0.012975,0.821987,0.870218,0.099928
2438,GSN_021414 = 1,"[3, 35, 41, 44, 47, 49, 53, 72, 88, 95, 102, 1...",0.100970,0.987023,0.012977,0.100898,0.106829,0.099937
2711,GSN_027413 = 1,"[3, 4, 11, 28, 30, 38, 40, 41, 42, 44, 52, 62,...",0.116085,0.986946,0.013054,0.115993,0.123548,0.100416


In [47]:
def jaccard_index (list_A : list, list_B : list) :
    intersection = len(list(set(list_A).intersection(list_B)))
    union = len(list(set(list_A).union(list_B)))
    return intersection / union


In [58]:
# Construction de l'arbre de granularité

def getGranularityTree(B_Granules : pd.DataFrame , info_table : pd.DataFrame) :
    from bigtree import Node
    root = Node('Root')
    covering_solution = set()
    remaining_objects = set(info_table.index)
    current_node = root
    while len(remaining_objects) > 0 :

        formules_determinantes = u_B_Granules[u_B_Granules['Entropy']==0].loc[:,'Formula']
        if len(formules_determinantes) == 0 :
            print('Impossible de progresser dans la granularité')
            break
        covering_solution.update(obj for granule in B_Granules[B_Granules['Entropy']==0].loc[:,'Granule'] for obj in granule)
        remaining_objects= remaining_objects - covering_solution

        for id in formules_determinantes.index :
                formule = formules_determinantes[id]
                Node(formule + f'\nClass = {1 if B_Granules.loc[id,'confidence_1'] == 1 else 0}', parent = current_node)

        B_Granules = B_Granules[B_Granules['Entropy']!=0]


        if len(B_Granules) == 0 :
            print('Toutes les granules ont été couvertes')
            break
        max_jaccard = 0
        max_jaccard_formula = None
        for i in B_Granules.index :
            jaccard = jaccard_index(B_Granules.loc[i,'Granule'], remaining_objects)
            if jaccard > max_jaccard :
                max_jaccard = jaccard
                max_jaccard_formula = B_Granules.loc[i,'Formula']

        new_node  = Node(max_jaccard_formula + '\nNoeud indéterminé', parent = current_node)
        current_node = new_node

        getGenerality(B_Granules, info_table)
        getConfidence(B_Granules, info_table)
        getCoverage(B_Granules, info_table)
        getEntropy(B_Granules, info_table)

        B_Granules.sort_values(by =['Entropy', 'Generality'], ascending = [True, False], inplace = True)
    
    if len(remaining_objects) == 0 :
        print('Tous les objets ont été couverts')
    else :
        max_jaccard = 0
        max_jaccard_granule = None
        max_jaccard_formula = None
        for i in B_Granules.index :
            jaccard = jaccard_index(B_Granules.loc[i,'Granule'], remaining_objects)
            if jaccard > max_jaccard :
                max_jaccard = jaccard
                max_jaccard_formula = B_Granules.loc[i,'Formula']

        new_node  = Node(max_jaccard_formula + '\nNoeud indéterminé', parent = current_node)
         
    return root



In [ ]:
from bigtree import tree_to_pillow
tree = getGranularityTree(u_B_Granules, u_info_table)
pillow = tree_to_pillow(tree)

pillow.save('granularity_tree.png')

pillow

  0%|          | 0/4066 [00:00<?, ?it/s]

  0%|          | 0/4066 [00:00<?, ?it/s]

  0%|          | 0/4066 [00:00<?, ?it/s]

In [ ]:
## Parcours de l'arbre de granularité
from bigtree import Node
def parcours_arbre(root: Node, info_table : pd.DataFrame) :
    y_hat = []
    for row in info_table.to_dict("records") :
        found = False
        node = root
        next_node = node
        while not found and next_node != None :
            next_node  = None
            for child in node.children :
                text = child.name
                formula, result = text.split('\n')
                if row[key.strip()] == value.strip() :
                    if result != 'Noeud indéterminé' :
                        key, value = formula.split('=')
                        y_hat.append(int(result.split('=')[1].strip()))
                        found = True
                        break
                    else : next_node = child
            node = next_node
        if not found :
            y_hat.append(None)
    
    return y_hat


y_hat = parcours_arbre(tree, test)
y = test['correct'].values

from sklearn.metrics import classification_report

print(classification_report(y, y_hat))


In [ ]:
# Comparaison avec un modèle de classification

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

X_train, y_train = train[attributes], train['correct']
X_test, y_test = test[attributes], test['correct']

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

print(classification_report(y_test, y_pred))


In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

print(classification_report(y_test, y_pred))